In [1]:
import numpy as np
import pandas as pd
import pickle
import re

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
from scipy import stats

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor
from xgboost import cv

import warnings
warnings.filterwarnings("ignore")

import funciones_modelos as fm

C:\Users\carlo\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
fh = open("C:/Users/carlo/Downloads/dataframe_final.pkl","rb")
data= pickle.load(fh)
fh.close()

Partimos con el dataset completo con 70 columnas que en dummies son casi 600 y con outliers y "ultra-outliers"




Se va a seleccionar el modelo en base a las siguientes decisiones:
1. ¿Qué hacemos con los outliers?
2. Tenemos muchas columnas, ¿qué variables son más relevantes y selecciono?
3. ¿Qué modelo concreto escoger?
4. ¿Qué hiperparámetros son los más adecuados?


In [4]:
# Previamente creo dos funciones para generar métricas personalizadas:

def npd(y_test,pred):
    totalCost = 0
    totalClocks = 0
    for i in range(len(y_test)):
        error = (y_test[i]-pred[i])
        if (y_test[i]-pred[i])/y_test[i] > 0.05:
            totalCost += error
            totalClocks += 1
            #print("TotalCost = {}".format(totalCost))
    return totalCost/totalClocks

In [5]:
def percentage(y_test,pred):
    totalClocks = 0
    for i in range(len(y_test)):
        error = (y_test[i]-pred[i])/y_test[i]
        if error > 0.05:
            totalClocks += 1
    return totalClocks/len(y_test)*100

--> Eliminamos desde el principio la variable Modelo ya que hay muchos modelos de los cuales solo hay 1 caso.

In [6]:
data = data.drop('Modelo', axis=1)

In [7]:
data= data.reset_index(drop=True)

# 1. Situación base: Todos los datos

## 1.1 Regresión múltiple: Caso Base

In [59]:
# Aplicamos una Regresión múltiple que es el modelo más simple para ver que tal funciona:


In [8]:
numericas = data[['Número de joyas', 'Resistente al agua', 'Perímetro', 'Reserva de marcha']]
categoricas = data.drop(['Precio', 'Número de joyas', 'Resistente al agua', 'Perímetro', 'Reserva de marcha'], axis=1)
precio = data['Precio']

In [9]:
dummies= pd.get_dummies(categoricas)

In [10]:
numericas1 = MinMaxScaler().fit_transform(numericas) 
numericas1 = pd.DataFrame(numericas1, columns = numericas.columns)

In [11]:
df= pd.concat([numericas1,dummies], axis=1).reset_index(drop=True)

In [12]:
X = df
y = precio

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

In [17]:
modelo = linear_model.LinearRegression()

In [18]:
modelo.fit(X_train,y_train)

LinearRegression()

In [19]:
# Métricas
yhat = modelo.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.exp(y_test),np.exp(yhat)))
mse = mean_squared_error(np.exp(y_test),np.exp(yhat))
smape = round(1/len(np.exp(y_test)) * np.sum( np.abs(np.exp(yhat)-np.exp(y_test)) / (np.abs(np.exp(y_test)) + np.abs(np.exp(yhat)))*100),2)
r2 = r2_score(np.exp(y_test), np.exp(yhat))
r2_ajustada = 1- (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)                  
npdMeanValue = npd(np.array(np.exp(y_test)),np.exp(yhat))
perValue = percentage(np.array(np.exp(y_test)),np.exp(yhat))

yhat_train = modelo.predict(X_train)
r2_train = r2_score(np.exp(y_train), np.exp(yhat_train))
r2_ajustada_train = 1- (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
mse_train = mean_squared_error(np.exp(y_train),np.exp(yhat_train))
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train),np.exp(yhat_train)))
smape_train = round(1/len(np.exp(y_train)) * np.sum( np.abs(np.exp(yhat_train)-np.exp(y_train)) / (np.abs(np.exp(y_train)) + np.abs(np.exp(yhat_train)))*100),2)
npdMeanValue_train = npd(np.array(np.exp(y_train)),np.exp(yhat_train))
perValue_train = percentage(np.array(np.exp(y_train)),np.exp(yhat_train))


print('Testing Performance')
print('MSE: {:.2f}'.format(mse))
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2_ajustada))
print('SMAPE:{:.2f}%'.format(smape))
print('NPD:{:.2f}'.format(npdMeanValue))
print('Percentage Undestimation:{:.2f}%'.format(perValue))

print("===============================")
print("Train Performance")
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
print('R2: {:.2f}'.format(r2_ajustada_train))
print('SMAPE:{:.2f}%'.format(smape_train))
print('NPD:{:.2f}'.format(npdMeanValue_train))
print('Percentage Undestimation:{:.2f}%'.format(perValue_train))

Testing Performance
MSE: 299546648837494669312.00
RMSE: 17307416006.95
R2: -20841119968.43
SMAPE:69.56%
NPD:1334662131.01
Percentage Undestimation:46.28%
Train Performance
MSE: 8518036158.16
RMSE: 92293.21
R2: 0.45
SMAPE:67.80%
NPD:25869.50
Percentage Undestimation:46.01%


In [ ]:
# Vemos que los resultados son muy poco útiles.

## 1.2 XGBoost: Caso Base

In [ ]:
# Para comprobar si el problema es que simplemente el modelo es excesivamente básico, vamos a probar un modelo de Boosting en el caso base

In [84]:
from xgboost import XGBRegressor
from xgboost import cv

In [85]:
numericas = data[['Número de joyas', 'Resistente al agua', 'Perímetro', 'Reserva de marcha']]
categoricas = data.drop(['Precio', 'Número de joyas', 'Resistente al agua', 'Perímetro', 'Reserva de marcha'], axis=1)
precio = data['Precio']

In [86]:
dummies= pd.get_dummies(categoricas)

In [87]:
sin_escalar = pd.concat([numericas,dummies],axis=1)

In [88]:
X = sin_escalar
y = precio

In [89]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

In [90]:
# Los hiperparámetros están escogidos por defecto -->  

In [91]:
param_grid = {"max_depth":    [8,10],
              "n_estimators": [500,600],
              "learning_rate": [0.01,0.15],}

In [92]:
regressor= XGBRegressor(eval_metric=['rmse'])

In [93]:
search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)

C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

The best hyperparameters are  {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 500}


In [94]:
xgboost=XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],)

In [95]:
xgboost= xgboost.fit(X_train, y_train)

In [96]:
# Métricas
yhat = xgboost.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.exp(y_test),np.exp(yhat)))
mse = mean_squared_error(np.exp(y_test),np.exp(yhat))
smape = round(1/len(np.exp(y_test)) * np.sum( np.abs(np.exp(yhat)-np.exp(y_test)) / (np.abs(np.exp(y_test)) + np.abs(np.exp(yhat)))*100),2)
r2 = r2_score(np.exp(y_test), np.exp(yhat))
r2_ajustada = 1- (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)                  
npdMeanValue = npd(np.array(np.exp(y_test)),np.exp(yhat))
perValue = percentage(np.array(np.exp(y_test)),np.exp(yhat))

yhat_train = xgboost.predict(X_train)
r2_train = r2_score(np.exp(y_train), np.exp(yhat_train))
r2_ajustada_train = 1- (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
mse_train = mean_squared_error(np.exp(y_train),np.exp(yhat_train))
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train),np.exp(yhat_train)))
smape_train = round(1/len(np.exp(y_train)) * np.sum( np.abs(np.exp(yhat_train)-np.exp(y_train)) / (np.abs(np.exp(y_train)) + np.abs(np.exp(yhat_train)))*100),2)
npdMeanValue_train = npd(np.array(np.exp(y_train)),np.exp(yhat_train))
perValue_train = percentage(np.array(np.exp(y_train)),np.exp(yhat_train))


print('Testing Performance')
print('MSE: {:.2f}'.format(mse))
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2_ajustada))
print('SMAPE:{:.2f}%'.format(smape))
print('NPD:{:.2f}'.format(npdMeanValue))
print('Percentage Undestimation:{:.2f}%'.format(perValue))

print("===============================")
print("Train Performance")
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
print('R2: {:.2f}'.format(r2_ajustada_train))
print('SMAPE:{:.2f}%'.format(smape_train))
print('NPD:{:.2f}'.format(npdMeanValue_train))
print('Percentage Undestimation:{:.2f}%'.format(perValue_train))


Testing Performance
MSE: 13789910109.01
RMSE: 117430.45
R2: 0.04
SMAPE:34.09%
NPD:18068.86
Percentage Undestimation:41.87%
Train Performance
MSE: 123503658.59
RMSE: 11113.22
R2: 0.99
SMAPE:31.34%
NPD:4704.61
Percentage Undestimation:40.20%


In [ ]:
# Vemos que el XGBoost también obtiene resultados muy pobres por lo que habrá que realizar un mayor tratamiento de los datos y hay overfit

# 2. Outliers, ¿Qué hacemos?

In [ ]:
# En el EDA se observa que hay Outliers y ultra-outliers y para tratarlos se pueden aplicar diferentes técnicas:
# 1. Aplicar una transformación logarítmica
# 2. Dejar fuera los datos más extremos
# 3. Crear clusters y aplicar modelos distintos

## 2.1 Transformación logarítmica

In [8]:
# Utilizamos un XGBoost como modelo base para ver si funciona la transformación logarítmica
# y ver si es suficiente como tratamiento de outliers

In [21]:
from xgboost import XGBRegressor
from xgboost import cv

In [22]:
numericas = data[['Número de joyas', 'Resistente al agua', 'Perímetro', 'Reserva de marcha']]
categoricas = data.drop(['Precio', 'Número de joyas', 'Resistente al agua', 'Perímetro', 'Reserva de marcha'], axis=1)
precio = data['Precio']

In [23]:
dummies= pd.get_dummies(categoricas)

In [24]:
sin_escalar = pd.concat([numericas,dummies],axis=1)

In [25]:
X = sin_escalar
y = np.log(precio)

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

In [28]:
param_grid = {"max_depth":    [10],
              "n_estimators": [500],
              "learning_rate": [0.01],}

In [29]:
regressor= XGBRegressor(eval_metric=['rmse'])

In [30]:
search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)

C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

The best hyperparameters are  {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 500}


In [31]:
xgboost=XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],)

In [32]:
xgboost= xgboost.fit(X_train, y_train)

In [33]:
# Métricas
yhat = cb.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.exp(y_test),np.exp(yhat)))
mse = mean_squared_error(np.exp(y_test),np.exp(yhat))
smape = round(1/len(np.exp(y_test)) * np.sum( np.abs(np.exp(yhat)-np.exp(y_test)) / (np.abs(np.exp(y_test)) + np.abs(np.exp(yhat)))*100),2)
r2 = r2_score(np.exp(y_test), np.exp(yhat))
r2_ajustada = 1- (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)                  
npdMeanValue = npd(np.array(np.exp(y_test)),np.exp(yhat))
perValue = percentage(np.array(np.exp(y_test)),np.exp(yhat))

yhat_train = cb.predict(X_train)
r2_train = r2_score(np.exp(y_train), np.exp(yhat_train))
r2_ajustada_train = 1- (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
mse_train = mean_squared_error(np.exp(y_train),np.exp(yhat_train))
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train),np.exp(yhat_train)))
smape_train = round(1/len(np.exp(y_train)) * np.sum( np.abs(np.exp(yhat_train)-np.exp(y_train)) / (np.abs(np.exp(y_train)) + np.abs(np.exp(yhat_train)))*100),2)
npdMeanValue_train = npd(np.array(np.exp(y_train)),np.exp(yhat_train))
perValue_train = percentage(np.array(np.exp(y_train)),np.exp(yhat_train))


print('Testing Performance')
print('MSE: {:.2f}'.format(mse))
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2_ajustada))
print('SMAPE:{:.2f}%'.format(smape))
print('NPD:{:.2f}'.format(npdMeanValue))
print('Percentage Undestimation:{:.2f}%'.format(perValue))

print("===============================")
print("Train Performance")
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
print('R2: {:.2f}'.format(r2_ajustada_train))
print('SMAPE:{:.2f}%'.format(smape_train))
print('NPD:{:.2f}'.format(npdMeanValue_train))
print('Percentage Undestimation:{:.2f}%'.format(perValue_train))

Testing Performance
MSE: 9012571180.18
RMSE: 94934.56
R2: 0.37
SMAPE:16.89%
NPD:21024.46
Percentage Undestimation:41.27%
Train Performance
MSE: 1968962278.28
RMSE: 44372.99
R2: 0.87
SMAPE:12.25%
NPD:10586.18
Percentage Undestimation:39.50%


In [ ]:
# Vemos que con la transformación logarítmica mejora el modelo pero no lo suficiente como para considerarlo útil
# Además sigue haciendo mucho overfit.

## 2.2 Optamos por quitar los outliers más extremos

In [ ]:
# Analizamos en qué punto de rango del precio podía ser interesante cortar para tener un modelo útil
# 100.000 nos parecía un punto óptimo por:
# 1. Nos quitamos solo el 4% de la muestra
# 2. La mayor parte de usuarios tendrán relojes por debajo de 100.000
# En definitiva: En el análisis exploratorio parecía el punto óptimo para hacer el corte: pérdida de datos vs modelo utilidad

## 2.2 XGBoost < 100.000€

In [47]:
from xgboost import XGBRegressor
from xgboost import cv

In [49]:
data = data.query('Precio < 100000')

In [51]:
numericas = data[['Número de joyas', 'Resistente al agua', 'Perímetro', 'Reserva de marcha']]
categoricas = data.drop(['Precio', 'Número de joyas', 'Resistente al agua', 'Perímetro', 'Reserva de marcha'], axis=1)
precio = data['Precio']

In [52]:
dummies= pd.get_dummies(categoricas)

In [53]:
sin_escalar = pd.concat([numericas,dummies],axis=1)

In [54]:
X = sin_escalar
y = precio

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

In [90]:
# Los hiperparámetros están escogidos por defecto -->  

In [56]:
param_grid = {"max_depth":    [10],
              "n_estimators": [500],
              "learning_rate": [0.01],}

In [57]:
regressor= XGBRegressor(eval_metric=['rmse'])

In [58]:
search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)

C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

The best hyperparameters are  {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 500}


In [59]:
xgboost=XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],)

In [60]:
xgboost= xgboost.fit(X_train, y_train)

In [61]:
# Métricas
yhat = cb.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.exp(y_test),np.exp(yhat)))
mse = mean_squared_error(np.exp(y_test),np.exp(yhat))
smape = round(1/len(np.exp(y_test)) * np.sum( np.abs(np.exp(yhat)-np.exp(y_test)) / (np.abs(np.exp(y_test)) + np.abs(np.exp(yhat)))*100),2)
r2 = r2_score(np.exp(y_test), np.exp(yhat))
r2_ajustada = 1- (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)                  
npdMeanValue = npd(np.array(np.exp(y_test)),np.exp(yhat))
perValue = percentage(np.array(np.exp(y_test)),np.exp(yhat))

yhat_train = cb.predict(X_train)
r2_train = r2_score(np.exp(y_train), np.exp(yhat_train))
r2_ajustada_train = 1- (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
mse_train = mean_squared_error(np.exp(y_train),np.exp(yhat_train))
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train),np.exp(yhat_train)))
smape_train = round(1/len(np.exp(y_train)) * np.sum( np.abs(np.exp(yhat_train)-np.exp(y_train)) / (np.abs(np.exp(y_train)) + np.abs(np.exp(yhat_train)))*100),2)
npdMeanValue_train = npd(np.array(np.exp(y_train)),np.exp(yhat_train))
perValue_train = percentage(np.array(np.exp(y_train)),np.exp(yhat_train))


print('Testing Performance')
print('MSE: {:.2f}'.format(mse))
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2_ajustada))
print('SMAPE:{:.2f}%'.format(smape))
print('NPD:{:.2f}'.format(npdMeanValue))
print('Percentage Undestimation:{:.2f}%'.format(perValue))

print("===============================")
print("Train Performance")
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
print('R2: {:.2f}'.format(r2_ajustada_train))
print('SMAPE:{:.2f}%'.format(smape_train))
print('NPD:{:.2f}'.format(npdMeanValue_train))
print('Percentage Undestimation:{:.2f}%'.format(perValue_train))

Testing Performance
MSE: 27364441.19
RMSE: 5231.10
R2: 0.89
SMAPE:24.81%
NPD:2945.38
Percentage Undestimation:37.90%
Train Performance
MSE: 5974014.85
RMSE: 2444.18
R2: 0.98
SMAPE:20.73%
NPD:1664.36
Percentage Undestimation:35.79%


In [ ]:
# En este caso vemos que quitando outliers la predicción mejora mucho.
# Sin embargo hay cierta diferencia entre train y test que puede indicar overfit
# ¿Puede haber variables que no aporten y por las cuales el modelo tiende a hacer overfit?

# 3. ¿Qué variables escoger? Reducir columnas

## 3.1 CatBoost con las variables para valorar importancia

In [28]:
data = data.query('Precio < 100000').reset_index(drop=True)

In [66]:
# Vemos que uno de los posibles problemas de que los modelos no salgan bien es que tenemos muchas variables X y además categóricas.
# En dummies llegan casi a las 600.
# Para seleccionar las variables vamos a relizar un catboost con todas las variables y valorar la importancia
# Se utiliza un CatBoost porque no necesita transformar las categóricas a dummies y resulta más sencillo valorar importancia

In [29]:
categoricas = data.drop(['Precio', 'Número de joyas', 'Resistente al agua', 'Perímetro', 'Reserva de marcha', 'Edad'], axis=1)
numericas = data[['Precio', 'Número de joyas', 'Resistente al agua', 'Perímetro', 'Reserva de marcha', 'Edad']]

In [30]:
df = pd.concat([categoricas,numericas],axis=1).reset_index(drop=True)

In [31]:
X = df.drop('Precio',axis=1)
y = df['Precio']

In [32]:
lista_categoricas= list(categoricas.columns)

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

In [42]:
cb = CatBoostRegressor(loss_function='RMSE',)
train_dataset = Pool(X_train, y_train, cat_features = lista_categoricas) 
test_dataset = Pool(X_test, y_test, cat_features = lista_categoricas)

In [43]:
grid = {'iterations': [200,400],
        'learning_rate': [0.01,0.05],
        'depth': [8],
        'l2_leaf_reg': [0.8]}


In [44]:
cb.grid_search(grid, train_dataset)

0:	learn: 17665.5106927	test: 17783.8702160	best: 17783.8702160 (0)	total: 169ms	remaining: 33.6s
1:	learn: 16998.8735456	test: 17100.6322145	best: 17100.6322145 (1)	total: 324ms	remaining: 32.1s
2:	learn: 16372.6938498	test: 16485.6352647	best: 16485.6352647 (2)	total: 505ms	remaining: 33.1s
3:	learn: 15767.3102338	test: 15869.4850812	best: 15869.4850812 (3)	total: 666ms	remaining: 32.7s
4:	learn: 15213.1574733	test: 15310.7618463	best: 15310.7618463 (4)	total: 851ms	remaining: 33.2s
5:	learn: 14672.6447423	test: 14778.4528557	best: 14778.4528557 (5)	total: 1.05s	remaining: 33.9s
6:	learn: 14161.9883391	test: 14263.4628398	best: 14263.4628398 (6)	total: 1.19s	remaining: 32.9s
7:	learn: 13703.8384547	test: 13799.5339295	best: 13799.5339295 (7)	total: 1.32s	remaining: 31.7s
8:	learn: 13245.4275901	test: 13329.3209316	best: 13329.3209316 (8)	total: 1.48s	remaining: 31.4s
9:	learn: 12806.5674695	test: 12910.8410029	best: 12910.8410029 (9)	total: 1.62s	remaining: 30.8s
10:	learn: 12400.785

{'params': {'depth': 8,
  'iterations': 200,
  'learning_rate': 0.05,
  'l2_leaf_reg': 0.8},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               

In [45]:
# Métricas
yhat = cb.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.exp(y_test),np.exp(yhat)))
mse = mean_squared_error(np.exp(y_test),np.exp(yhat))
smape = round(1/len(np.exp(y_test)) * np.sum( np.abs(np.exp(yhat)-np.exp(y_test)) / (np.abs(np.exp(y_test)) + np.abs(np.exp(yhat)))*100),2)
r2 = r2_score(np.exp(y_test), np.exp(yhat))
r2_ajustada = 1- (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)                  
npdMeanValue = npd(np.array(np.exp(y_test)),np.exp(yhat))
perValue = percentage(np.array(np.exp(y_test)),np.exp(yhat))

yhat_train = cb.predict(X_train)
r2_train = r2_score(np.exp(y_train), np.exp(yhat_train))
r2_ajustada_train = 1- (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
mse_train = mean_squared_error(np.exp(y_train),np.exp(yhat_train))
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train),np.exp(yhat_train)))
smape_train = round(1/len(np.exp(y_train)) * np.sum( np.abs(np.exp(yhat_train)-np.exp(y_train)) / (np.abs(np.exp(y_train)) + np.abs(np.exp(yhat_train)))*100),2)
npdMeanValue_train = npd(np.array(np.exp(y_train)),np.exp(yhat_train))
perValue_train = percentage(np.array(np.exp(y_train)),np.exp(yhat_train))


print('Testing Performance')
print('MSE: {:.2f}'.format(mse))
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2_ajustada))
print('SMAPE:{:.2f}%'.format(smape))
print('NPD:{:.2f}'.format(npdMeanValue))
print('Percentage Undestimation:{:.2f}%'.format(perValue))

print("===============================")
print("Train Performance")
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
print('R2: {:.2f}'.format(r2_ajustada_train))
print('SMAPE:{:.2f}%'.format(smape_train))
print('NPD:{:.2f}'.format(npdMeanValue_train))
print('Percentage Undestimation:{:.2f}%'.format(perValue_train))

Testing Performance
MSE: 28882223.22
RMSE: 5374.22
R2: 0.89
SMAPE:22.51%
NPD:3789.87
Percentage Undestimation:32.85%
Train Performance
MSE: 21009593.84
RMSE: 4583.62
R2: 0.92
SMAPE:22.28%
NPD:3157.20
Percentage Undestimation:33.87%


In [46]:
# Estas son las definitivas y cogeremos las que tienen una importancia > 2%
len(X.columns)
len(cb.feature_importances_)
importances=pd.DataFrame([X.columns,cb.feature_importances_], index=["feature","importance"]).T
print(importances.sort_values("importance", ascending = False).head(30))

                              feature importance
0                               Marca  44.945747
65                          Perímetro    6.14718
11                Material del cierre   5.609543
3             Contenido de la entrega   5.501311
63                    Número de joyas    4.40654
14                    Tipo de Calibre   4.013353
12                Material de la caja   3.840269
66                  Reserva de marcha   2.829945
64                 Resistente al agua   2.599802
2                              Estado   2.463111
1              Material de la pulsera   2.097178
8                              Esfera   2.090696
15                         Cronógrafo   1.699941
9                  Esfera con números   1.662061
10                Color de la pulsera   1.646521
6                  Material del bisel   1.499163
67                               Edad   1.011818
7                             Cristal    1.00869
27                Calendario perpetuo   0.807632
51  Con piedras prec

## 3.2 XGBoost con variables relevantes

In [62]:
from xgboost import XGBRegressor
from xgboost import cv

In [63]:
data = data.query('Precio < 100000')

In [64]:
categoricas = data[['Marca', 'Material del cierre', 'Contenido de la entrega', 'Tipo de Calibre', 'Material de la caja', 'Estado', 'Material de la pulsera', 'Esfera']]
numericas = data[['Perímetro', 'Número de joyas', 'Reserva de marcha', 'Resistente al agua']]
precio = data['Precio']

In [65]:
dummies= pd.get_dummies(categoricas)

In [66]:
sin_escalar = pd.concat([numericas,dummies],axis=1)

In [67]:
X = sin_escalar
y = precio

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

In [69]:
param_grid = {"max_depth":    [10],
              "n_estimators": [500],
              "learning_rate": [0.01],}

In [70]:
regressor= XGBRegressor(eval_metric=['rmse'])

In [71]:
search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)

C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

The best hyperparameters are  {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 500}


In [72]:
xgboost=XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],)

In [73]:
xgboost= xgboost.fit(X_train, y_train)

In [74]:
# Métricas
yhat = xgboost.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.exp(y_test),np.exp(yhat)))
mse = mean_squared_error(np.exp(y_test),np.exp(yhat))
smape = round(1/len(np.exp(y_test)) * np.sum( np.abs(np.exp(yhat)-np.exp(y_test)) / (np.abs(np.exp(y_test)) + np.abs(np.exp(yhat)))*100),2)
r2 = r2_score(np.exp(y_test), np.exp(yhat))
r2_ajustada = 1- (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)                  
npdMeanValue = npd(np.array(np.exp(y_test)),np.exp(yhat))
perValue = percentage(np.array(np.exp(y_test)),np.exp(yhat))

yhat_train = xgboost.predict(X_train)
r2_train = r2_score(np.exp(y_train), np.exp(yhat_train))
r2_ajustada_train = 1- (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
mse_train = mean_squared_error(np.exp(y_train),np.exp(yhat_train))
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train),np.exp(yhat_train)))
smape_train = round(1/len(np.exp(y_train)) * np.sum( np.abs(np.exp(yhat_train)-np.exp(y_train)) / (np.abs(np.exp(y_train)) + np.abs(np.exp(yhat_train)))*100),2)
npdMeanValue_train = npd(np.array(np.exp(y_train)),np.exp(yhat_train))
perValue_train = percentage(np.array(np.exp(y_train)),np.exp(yhat_train))


print('Testing Performance')
print('MSE: {:.2f}'.format(mse))
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2_ajustada))
print('SMAPE:{:.2f}%'.format(smape))
print('NPD:{:.2f}'.format(npdMeanValue))
print('Percentage Undestimation:{:.2f}%'.format(perValue))

print("===============================")
print("Train Performance")
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
print('R2: {:.2f}'.format(r2_ajustada_train))
print('SMAPE:{:.2f}%'.format(smape_train))
print('NPD:{:.2f}'.format(npdMeanValue_train))
print('Percentage Undestimation:{:.2f}%'.format(perValue_train))

Testing Performance
MSE: 29305808.86
RMSE: 5413.48
R2: 0.89
SMAPE:24.13%
NPD:3126.80
Percentage Undestimation:37.99%
Train Performance
MSE: 11415204.67
RMSE: 3378.64
R2: 0.96
SMAPE:21.23%
NPD:2099.18
Percentage Undestimation:35.86%


In [ ]:
# Este modelo solo con las columnas más relevantes sigue teniendo métricas similares al modelo con todas las columnas
# Parece que aún hay cierto overfit

# 4. ¿Qué modelo escogemos?  4 modelos con las condiciones anteriores

## 4.1 XGBoost --> Ya está aplicado en 3.2

## 4.2 Regresión múltiple

In [76]:
categoricas = data[['Marca', 'Material del cierre', 'Contenido de la entrega', 'Tipo de Calibre', 'Material de la caja', 'Estado', 'Material de la pulsera', 'Esfera']]
numericas = data[['Perímetro', 'Número de joyas', 'Reserva de marcha', 'Resistente al agua']]
precio = data['Precio']

In [77]:
dummies= pd.get_dummies(categoricas)

In [78]:
numericas1 = MinMaxScaler().fit_transform(numericas) 
numericas1 = pd.DataFrame(numericas1, columns = numericas.columns)

In [79]:
df= pd.concat([numericas1,dummies], axis=1).reset_index(drop=True)

In [80]:
X = df
y = precio

In [82]:
modelo = linear_model.LinearRegression()

In [83]:
modelo.fit(X_train,y_train)

LinearRegression()

In [84]:
# Métricas
yhat = modelo.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.exp(y_test),np.exp(yhat)))
mse = mean_squared_error(np.exp(y_test),np.exp(yhat))
smape = round(1/len(np.exp(y_test)) * np.sum( np.abs(np.exp(yhat)-np.exp(y_test)) / (np.abs(np.exp(y_test)) + np.abs(np.exp(yhat)))*100),2)
r2 = r2_score(np.exp(y_test), np.exp(yhat))
r2_ajustada = 1- (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)                  
npdMeanValue = npd(np.array(np.exp(y_test)),np.exp(yhat))
perValue = percentage(np.array(np.exp(y_test)),np.exp(yhat))

yhat_train = modelo.predict(X_train)
r2_train = r2_score(np.exp(y_train), np.exp(yhat_train))
r2_ajustada_train = 1- (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
mse_train = mean_squared_error(np.exp(y_train),np.exp(yhat_train))
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train),np.exp(yhat_train)))
smape_train = round(1/len(np.exp(y_train)) * np.sum( np.abs(np.exp(yhat_train)-np.exp(y_train)) / (np.abs(np.exp(y_train)) + np.abs(np.exp(yhat_train)))*100),2)
npdMeanValue_train = npd(np.array(np.exp(y_train)),np.exp(yhat_train))
perValue_train = percentage(np.array(np.exp(y_train)),np.exp(yhat_train))


print('Testing Performance')
print('MSE: {:.2f}'.format(mse))
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2_ajustada))
print('SMAPE:{:.2f}%'.format(smape))
print('NPD:{:.2f}'.format(npdMeanValue))
print('Percentage Undestimation:{:.2f}%'.format(perValue))

print("===============================")
print("Train Performance")
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
print('R2: {:.2f}'.format(r2_ajustada_train))
print('SMAPE:{:.2f}%'.format(smape_train))
print('NPD:{:.2f}'.format(npdMeanValue_train))
print('Percentage Undestimation:{:.2f}%'.format(perValue_train))

Testing Performance
MSE: 81148737.66
RMSE: 9008.26
R2: 0.69
SMAPE:49.20%
NPD:6228.46
Percentage Undestimation:41.45%
Train Performance
MSE: 76758226.63
RMSE: 8761.18
R2: 0.70
SMAPE:49.02%
NPD:6033.03
Percentage Undestimation:41.98%


In [ ]:
# Vemos que el propio modelo de regresión mejora considerablemente

## 4.3 CatBoost

In [85]:
categoricas = data[['Marca', 'Material del cierre', 'Contenido de la entrega', 'Tipo de Calibre', 'Material de la caja', 'Estado', 'Material de la pulsera', 'Esfera']]
numericas = data[['Perímetro', 'Número de joyas', 'Reserva de marcha', 'Resistente al agua']]
precio = data['Precio']

In [86]:
df = pd.concat([categoricas,numericas],axis=1).reset_index(drop=True)

In [88]:
X = df
y = precio

In [89]:
lista_categoricas= list(categoricas.columns)

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

In [93]:
cb = CatBoostRegressor(loss_function='RMSE',)
train_dataset = Pool(X_train, y_train, cat_features = lista_categoricas) 
test_dataset = Pool(X_test, y_test, cat_features = lista_categoricas)

In [94]:
grid = {'iterations': [200],
        'learning_rate': [0.05],
        'depth': [8],
        'l2_leaf_reg': [0.8]}


In [95]:
cb.grid_search(grid, train_dataset)

0:	learn: 17667.3904857	test: 17798.0584436	best: 17798.0584436 (0)	total: 137ms	remaining: 27.3s
1:	learn: 17018.7393374	test: 17143.9376430	best: 17143.9376430 (1)	total: 316ms	remaining: 31.3s
2:	learn: 16414.3810815	test: 16550.1146954	best: 16550.1146954 (2)	total: 447ms	remaining: 29.4s
3:	learn: 15852.3222610	test: 15967.5261720	best: 15967.5261720 (3)	total: 603ms	remaining: 29.5s
4:	learn: 15285.5794532	test: 15409.1469520	best: 15409.1469520 (4)	total: 745ms	remaining: 29.1s
5:	learn: 14752.9941169	test: 14863.6865169	best: 14863.6865169 (5)	total: 886ms	remaining: 28.6s
6:	learn: 14245.3775328	test: 14355.0667923	best: 14355.0667923 (6)	total: 1.04s	remaining: 28.8s
7:	learn: 13801.0623655	test: 13894.7887223	best: 13894.7887223 (7)	total: 1.12s	remaining: 27s
8:	learn: 13330.5189106	test: 13434.1784542	best: 13434.1784542 (8)	total: 1.26s	remaining: 26.7s
9:	learn: 12883.5759812	test: 12993.6329618	best: 12993.6329618 (9)	total: 1.4s	remaining: 26.6s
10:	learn: 12494.371081

{'params': {'depth': 8,
  'iterations': 200,
  'learning_rate': 0.05,
  'l2_leaf_reg': 0.8},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               

In [96]:
# Métricas
yhat = cb.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.exp(y_test),np.exp(yhat)))
mse = mean_squared_error(np.exp(y_test),np.exp(yhat))
smape = round(1/len(np.exp(y_test)) * np.sum( np.abs(np.exp(yhat)-np.exp(y_test)) / (np.abs(np.exp(y_test)) + np.abs(np.exp(yhat)))*100),2)
r2 = r2_score(np.exp(y_test), np.exp(yhat))
r2_ajustada = 1- (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)                  
npdMeanValue = npd(np.array(np.exp(y_test)),np.exp(yhat))
perValue = percentage(np.array(np.exp(y_test)),np.exp(yhat))

yhat_train = cb.predict(X_train)
r2_train = r2_score(np.exp(y_train), np.exp(yhat_train))
r2_ajustada_train = 1- (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
mse_train = mean_squared_error(np.exp(y_train),np.exp(yhat_train))
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train),np.exp(yhat_train)))
smape_train = round(1/len(np.exp(y_train)) * np.sum( np.abs(np.exp(yhat_train)-np.exp(y_train)) / (np.abs(np.exp(y_train)) + np.abs(np.exp(yhat_train)))*100),2)
npdMeanValue_train = npd(np.array(np.exp(y_train)),np.exp(yhat_train))
perValue_train = percentage(np.array(np.exp(y_train)),np.exp(yhat_train))


print('Testing Performance')
print('MSE: {:.2f}'.format(mse))
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2_ajustada))
print('SMAPE:{:.2f}%'.format(smape))
print('NPD:{:.2f}'.format(npdMeanValue))
print('Percentage Undestimation:{:.2f}%'.format(perValue))

print("===============================")
print("Train Performance")
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
print('R2: {:.2f}'.format(r2_ajustada_train))
print('SMAPE:{:.2f}%'.format(smape_train))
print('NPD:{:.2f}'.format(npdMeanValue_train))
print('Percentage Undestimation:{:.2f}%'.format(perValue_train))

Testing Performance
MSE: 32811837.34
RMSE: 5728.16
R2: 0.87
SMAPE:22.76%
NPD:3995.72
Percentage Undestimation:32.85%
Train Performance
MSE: 24713449.73
RMSE: 4971.26
R2: 0.90
SMAPE:21.98%
NPD:3479.52
Percentage Undestimation:32.95%


## 4.4 LightGBM

In [97]:
import lightgbm as lgb

In [98]:
categoricas = data[['Marca', 'Material del cierre', 'Contenido de la entrega', 'Tipo de Calibre', 'Material de la caja', 'Estado', 'Material de la pulsera', 'Esfera']]
numericas = data[['Perímetro', 'Número de joyas', 'Reserva de marcha', 'Resistente al agua']]
precio = data['Precio']

In [99]:
dummies= pd.get_dummies(categoricas)

In [100]:
sin_escalar = pd.concat([numericas,dummies],axis=1)


In [101]:
### Se quitan caracteres especiales en el df porque sino da un error en la lectura

In [102]:
import re
sin_escalar = sin_escalar.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x)).reset_index(drop=True)

In [103]:
X = sin_escalar
y = precio

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

In [105]:
param_grid = {"max_depth":    [8],
              "n_estimators": [400],
              "learning_rate": [0.01],
                "metric": ['rmse']}

In [106]:
#regressor= lgb.LGBMRegressor(eval_metric=['rmse'])

In [107]:
search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)

C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:01:20] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "metric" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:01:24] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "metric" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


...

[20:01:37] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "metric" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:01:44] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "metric" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:01:51] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "metric" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


The best hyperparameters are  {'learning_rate': 0.01, 'max_depth': 8, 'metric': 'rmse', 'n_estimators': 400}


In [108]:
regressor=lgb.LGBMRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],)

In [109]:
lightgbm= regressor.fit(X_train, y_train)

In [111]:
# Métricas
yhat = lightgbm.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.exp(y_test),np.exp(yhat)))
mse = mean_squared_error(np.exp(y_test),np.exp(yhat))
smape = round(1/len(np.exp(y_test)) * np.sum( np.abs(np.exp(yhat)-np.exp(y_test)) / (np.abs(np.exp(y_test)) + np.abs(np.exp(yhat)))*100),2)
r2 = r2_score(np.exp(y_test), np.exp(yhat))
r2_ajustada = 1- (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)                  
npdMeanValue = npd(np.array(np.exp(y_test)),np.exp(yhat))
perValue = percentage(np.array(np.exp(y_test)),np.exp(yhat))

yhat_train = lightgbm.predict(X_train)
r2_train = r2_score(np.exp(y_train), np.exp(yhat_train))
r2_ajustada_train = 1- (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
mse_train = mean_squared_error(np.exp(y_train),np.exp(yhat_train))
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train),np.exp(yhat_train)))
smape_train = round(1/len(np.exp(y_train)) * np.sum( np.abs(np.exp(yhat_train)-np.exp(y_train)) / (np.abs(np.exp(y_train)) + np.abs(np.exp(yhat_train)))*100),2)
npdMeanValue_train = npd(np.array(np.exp(y_train)),np.exp(yhat_train))
perValue_train = percentage(np.array(np.exp(y_train)),np.exp(yhat_train))


print('Testing Performance')
print('MSE: {:.2f}'.format(mse))
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2_ajustada))
print('SMAPE:{:.2f}%'.format(smape))
print('NPD:{:.2f}'.format(npdMeanValue))
print('Percentage Undestimation:{:.2f}%'.format(perValue))

print("===============================")
print("Train Performance")
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
print('R2: {:.2f}'.format(r2_ajustada_train))
print('SMAPE:{:.2f}%'.format(smape_train))
print('NPD:{:.2f}'.format(npdMeanValue_train))
print('Percentage Undestimation:{:.2f}%'.format(perValue_train))

Testing Performance
MSE: 36099229.65
RMSE: 6008.26
R2: 0.86
SMAPE:26.52%
NPD:5262.31
Percentage Undestimation:26.34%
Train Performance
MSE: 31788605.99
RMSE: 5638.14
R2: 0.88
SMAPE:26.25%
NPD:4927.60
Percentage Undestimation:25.79%


# 5. Selección del mejor modelo y transformación logarítmica

In [8]:
from xgboost import XGBRegressor
from xgboost import cv

In [11]:
data = data.query('Precio < 100000')

In [13]:
categoricas = data[['Marca', 'Material del cierre', 'Contenido de la entrega', 'Tipo de Calibre', 'Material de la caja', 'Estado', 'Material de la pulsera', 'Esfera']]
numericas = data[['Perímetro', 'Número de joyas', 'Reserva de marcha', 'Resistente al agua']]
precio = data['Precio']

In [14]:
dummies= pd.get_dummies(categoricas)

In [15]:
sin_escalar = pd.concat([numericas,dummies],axis=1)

In [16]:
X = sin_escalar
y = np.log(precio)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

In [18]:
param_grid = {"max_depth":    [10],
              "n_estimators": [500],
              "learning_rate": [0.01],}

In [19]:
regressor= XGBRegressor(eval_metric=['rmse'])

In [20]:
search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)

C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\carlo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

The best hyperparameters are  {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 500}


In [21]:
xgboost=XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],)

In [22]:
xgboost= xgboost.fit(X_train, y_train)

In [24]:
# Métricas
yhat = xgboost.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.exp(y_test),np.exp(yhat)))
mse = mean_squared_error(np.exp(y_test),np.exp(yhat))
smape = round(1/len(np.exp(y_test)) * np.sum( np.abs(np.exp(yhat)-np.exp(y_test)) / (np.abs(np.exp(y_test)) + np.abs(np.exp(yhat)))*100),2)
r2 = r2_score(np.exp(y_test), np.exp(yhat))
r2_ajustada = 1- (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)                  
npdMeanValue = npd(np.array(np.exp(y_test)),np.exp(yhat))
perValue = percentage(np.array(np.exp(y_test)),np.exp(yhat))

yhat_train = xgboost.predict(X_train)
r2_train = r2_score(np.exp(y_train), np.exp(yhat_train))
r2_ajustada_train = 1- (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
mse_train = mean_squared_error(np.exp(y_train),np.exp(yhat_train))
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train),np.exp(yhat_train)))
smape_train = round(1/len(np.exp(y_train)) * np.sum( np.abs(np.exp(yhat_train)-np.exp(y_train)) / (np.abs(np.exp(y_train)) + np.abs(np.exp(yhat_train)))*100),2)
npdMeanValue_train = npd(np.array(np.exp(y_train)),np.exp(yhat_train))
perValue_train = percentage(np.array(np.exp(y_train)),np.exp(yhat_train))


print('Testing Performance')
print('MSE: {:.2f}'.format(mse))
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2_ajustada))
print('SMAPE:{:.2f}%'.format(smape))
print('NPD:{:.2f}'.format(npdMeanValue))
print('Percentage Undestimation:{:.2f}%'.format(perValue))

print("===============================")
print("Train Performance")
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
print('R2: {:.2f}'.format(r2_ajustada_train))
print('SMAPE:{:.2f}%'.format(smape_train))
print('NPD:{:.2f}'.format(npdMeanValue_train))
print('Percentage Undestimation:{:.2f}%'.format(perValue_train))
           

Testing Performance
MSE: 30250088.51
RMSE: 5500.01
R2: 0.88
SMAPE:17.25%
NPD:3294.86
Percentage Undestimation:42.22%
Train Performance
MSE: 21198467.32
RMSE: 4604.18
R2: 0.92
SMAPE:13.66%
NPD:2835.41
Percentage Undestimation:39.29%


In [ ]:
# En este modelo en logaritmo solo podemos utilizar SMAPE para comparar y vemos que el modelo funciona mejor

In [ ]:
# Puntos de mejora:
# - Mejor comprensión de los outliers y posible creación de modelos para ellos
# - Aplicación de CrossValidation para garantizar que las métricas son estables
# - Optimizar los hiperparámetros de los modelos